To make this work

Setup the environment variables in a terminal, sourcing the necessary .bashrc and install/setup.bash

Then launch a server

Then attach this notebook to it

extend the python path with the conda env site packages

In [11]:
!source /home/richard/ar4_ros_driver/install/setup.bash

In [2]:
#! /usr/bin/env python3
import sys

# sys.path.append("/opt/ros/humble/lib/python3.10/site-packages")
# sys.path.append("/opt/ros/humble/local/lib/python3.10/dist-packages")
# sys.path.append("~/ar4_ros_driver/install/ros2_aruco_interfaces/local/lib/python3.10/dist-packages")

import rclpy
from rclpy.node import Node

from geometry_msgs.msg import TransformStamped
from ros2_aruco_interfaces.msg import ArucoMarkers
from tf2_ros import TransformBroadcaster

In [4]:
sys.path.append("/home/richard/miniconda3/envs/rgbd/lib/python3.10/site-packages")
import torch

In [5]:
class CalibrationArucoPublisher(Node):
    """ROS2 node that listens to the aruco markers topic and publishes the 
    transform of the specific aruco marker for calibration to tf2.
    """

    def __init__(self):
        super().__init__("calibration_aruco_publisher")

        # ID of the aruco marker mounted on the robot
        self.marker_id = self.declare_parameter(
            "marker_id", 5).get_parameter_value().integer_value #test id 0

        self.tf_broadcaster = TransformBroadcaster(self)
        self.subscription = self.create_subscription(ArucoMarkers,
                                                     "/aruco_markers",
                                                     self.handle_aruco_markers,
                                                     1)

    def handle_aruco_markers(self, msg: ArucoMarkers):
        cal_marker_pose = None
        for i, marker_id in enumerate(msg.marker_ids):
            if marker_id == self.marker_id:
                cal_marker_pose = msg.poses[i]
                break

        if cal_marker_pose is None:
            return

        t = TransformStamped()
        t.header.stamp = self.get_clock().now().to_msg()
        t.header.frame_id = "/camera_color_optical_frame"
        t.child_frame_id = "/calibration_aruco"

        t.transform.translation.x = cal_marker_pose.position.x
        t.transform.translation.y = cal_marker_pose.position.y
        t.transform.translation.z = cal_marker_pose.position.z
        t.transform.rotation = cal_marker_pose.orientation

        # Send the transformation
        self.tf_broadcaster.sendTransform(t)

In [6]:
def main():
    rclpy.init()
    node = CalibrationArucoPublisher()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass

    rclpy.shutdown()

In [7]:
torch.__version__

'2.4.0+cu121'

In [8]:
rclpy.init()
node = CalibrationArucoPublisher()
try:
  rclpy.spin(node)
except KeyboardInterrupt:
  pass

rclpy.shutdown()

RCLError: failed to shutdown: rcl_shutdown already called on the given context, at ./src/rcl/init.c:241

In [16]:
# rclpy.init()
node = CalibrationArucoPublisher()

[WARN] [1725242957.927260108] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


In [17]:
rclpy.spin_once(node)

RCLError: failed to initialize wait set: the given context is not valid, either rcl_init() was not called or rcl_shutdown() was called., at ./src/rcl/wait.c:130